In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import LeaveOneOut, StratifiedKFold, train_test_split
from hyperopt import hp,STATUS_OK,Trials,fmin,tpe
from hyperopt.early_stop import no_progress_loss
import warnings
from copy import deepcopy
warnings.filterwarnings("ignore")

1. data preparation

In [2]:
# --------------------------------------------train set-----------------------------
data_1 = pd.read_excel('path to the file ')
data_1 = data_1[data_1["AGE"] >= 18]
data_1['F'] = ((data_1['L_F'] == 1)|(data_1['R_F'] == 1)) + 0
data_1['T'] = ((data_1['L_T'] == 1)|(data_1['R_T'] == 1)) + 0
data_1['P'] = ((data_1['L_P'] == 1)|(data_1['R_P'] == 1)) + 0
data_1['O'] = ((data_1['L_O'] == 1)|(data_1['R_O'] == 1)) + 0
data_1['I'] = ((data_1['L_I'] == 1)|(data_1['R_I'] == 1)) + 0
data_1['CC'] = ((data_1['L_CC'] == 1)|(data_1['R_CC'] == 1)) + 0
data_1['BG'] = ((data_1['L_BG'] == 1)|(data_1['R_BG'] == 1)) + 0
data_1['TH'] = ((data_1['L_TH'] == 1)|(data_1['R_TH'] == 1)) + 0
data_1['M'] = ((data_1['L_M'] == 1)|(data_1['R_M'] == 1)) + 0
data_1['S'] = ((data_1['L_S'] == 1)|(data_1['R_S'] == 1)) + 0
data_1['H'] = ((data_1['L_H'] == 1)|(data_1['R_H'] == 1)) + 0
data_1['GC'] = ((data_1['L_GC'] == 1)|(data_1['R_GC'] == 1)) + 0
data_1['BILATERAL'] = ((data_1['L'] + data_1['R']) == 2) + 0
loco_lob = ['F', 'T', 'P', 'O', 'I', 'CC', 'BG', 'TH']
loco_lr = ['F', 'T', 'P', 'O', 'I', 'CC', 'BG', 'TH', 
         'M', 'S', 'H', 'GC']
loco = ['L_F', 'L_T', 'L_P', 'L_O', 'L_I', 'L_CC', 'L_BG', 'L_TH', 
      'L_M', 'L_S', 'L_H', 'L_GC',
      'R_F', 'R_T', 'R_P', 'R_O', 'R_I', 'R_CC', 'R_BG', 'R_TH', 
      'R_M', 'R_S', 'R_H', 'R_GC'] 
loco_a = ['L_F', 'L_T', 'L_P', 'L_O', 'L_I', 'L_CC', 'L_BG', 'L_TH',
          'L_M', 'L_S', 'L_H', 'L_GC',
        'R_F', 'R_T', 'R_P', 'R_O', 'R_I', 'R_CC', 'R_BG', 'R_TH',
          'R_M', 'R_S', 'R_H', 'R_GC', 
        'F', 'T', 'P', 'O', 'I', 'CC', 'BG', 'TH',
          'M', 'S', 'H', 'GC']
#sym
symp_p = ['S_I_P', 'S_M_P', 'S_S_P', 'S_Ap_P', 'S_Cog_P', 'S_BTH_P', 'S_E_P',
          'S_Con_P', 'S_Ab_P', 'S_V_P', 'S_Au_P', ]
symp_s = ['S_I', 'S_M', 'S_S', 'S_Ap', 'S_Cog', 'S_BTH', 'S_E',
          'S_Con', 'S_Ab', 'S_V', 'S_Au']
symp_p_only = []
[symp_p_only.append(sy + '_P_ONLY') for sy in symp_p]
symp_s_only = []
[symp_s_only.append(sy + '_S_ONLY') for sy in symp_s]
##sym_t
symp_t = ['S_T', 'S_ST', 'S_DT', 'S_ET', 'S_ST_RA', 'S_DT_RA', 'S_ET_RA']
symp_st = ['S_I_DIS_T', 'S_I_FOLL_T', 'S_M_T', 'S_S_T', 'S_AP_T', 'S_COG_T', 'S_BTH_T', 'S_E_T',
           'S_CON_T', 'S_ABD_T', 'S_V_T', 'S_AUD_T']
symp_st_ra = []
[symp_st_ra.append((sy_st + '_RA')) for sy_st in symp_st]
symp_st_a = symp_st + symp_st_ra

#  feature engineering
data_1['NUM_L'] = 0
data_1['LOCALIZED'] = 0
data_1['S_NUM_P'] = 0
data_1['S_NUM_S'] = 0
data_1['S_P_ONLY'] = 0
data_1['S_S_ONLY'] = 0
data_1['S_I'] = data_1['S_I_P']
data_1['S_ST_RA'] = 0
data_1['S_DT_RA'] = 0
data_1['S_ET_RA'] = 0
for sy in symp_st:
    data_1[sy + '_RA'] = 0
for i in range(len(data_1)):
    # num_lobs       
    for lo in loco_lob:
        if data_1[lo][i]>0:
            data_1.loc[i, 'NUM_L'] += 1
    # localized (num_l = 1)
    if data_1.loc[i, 'NUM_L'] == 1:
        data_1.loc[i, 'LOCALIZED'] = 1
    # sy_t_ratio  
    for sy in symp_st:
        data_1.loc[i, sy+'_RA'] = data_1[sy][i] / data_1['S_T'][i]
    #ST_RA
    data_1.loc[i, 'S_ST_RA'] = data_1['S_ST'][i] / data_1['S_T'][i]
    data_1.loc[i, 'S_DT_RA'] = data_1['S_DT'][i] / data_1['S_T'][i]
    data_1.loc[i, 'S_ET_RA'] = data_1['S_ET'][i] / data_1['S_T'][i]
    # sy_num
    for sy_p in symp_p:
        data_1.loc[i, 'S_NUM_P'] += data_1[sy_p][i]                
    for sy_s in symp_s:
        data_1.loc[i, 'S_NUM_S'] += data_1[sy_s][i]
    # sym_only******
    data_1.loc[i, 'S_P_ONLY'] = 1 - (data_1['S_NUM_P'][i] > 1)           
    for sy in symp_p:
        data_1.loc[i, sy + '_P_ONLY'] = data_1[sy][i] * data_1['S_P_ONLY'][i]
          
    data_1.loc[i, 'S_S_ONLY'] = 1 - (data_1['S_NUM_S'][i] > 1)              
    for sy in symp_s:
        data_1.loc[i, sy + '_S_ONLY'] = data_1[sy][i] * data_1['S_S_ONLY'][i]  
data_1 = data_1.drop('S_I',axis = 1)
data_1

,ID,SEX,AGE,L,R,L_F,L_T,L_P,L_O,L_I,...,S_M_S_ONLY,S_S_S_ONLY,S_Ap_S_ONLY,S_Cog_S_ONLY,S_BTH_S_ONLY,S_E_S_ONLY,S_Con_S_ONLY,S_Ab_S_ONLY,S_V_S_ONLY,S_Au_S_ONLY
0,1,1,42,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1,40,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,3,1,47,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1,71,1,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1,46,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,986,1,49,1,0,0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
986,987,1,45,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
987,988,2,34,1,0,1,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
988,989,2,45,1,0,1,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [3]:
def hyperopt_xgb(params):
    auc_i = []
    for n in range(100):
        x_train, x_val, y_train, y_val=train_test_split(train_data, train_label, train_size = 0.8, random_state= n, stratify= train_label)
        clf = XGBClassifier(n_estimators = int(params['n_estimators']), max_depth = int(params['max_depth']), 
                            learning_rate = params['learning_rate'], min_child_weight = int(params['min_child_weight']),
                            gamma = params['gamma'], subsample = params['subsample'],
                            random_state = 123)
        clf.fit(x_train, y_train)
        probs = clf.predict_proba(x_val)
        auc_n = metrics.roc_auc_score(y_val, probs[:, 1])
        auc_i.append(auc_n)
    auc_m = np.mean(auc_i)
    return -auc_m 

def param_hyperopt(max_eval=100):
    trials = Trials()
    # early_stop_f = no_progress_loss(100)
    params_best = fmin(hyperopt_xgb,
                       space = space4xgc,
                       max_evals = max_eval,
                       algo =tpe.suggest,
                       verbose = True,
                       trials = trials)
                       #early_stop_fn = early_stop_f)
    print('\n','\n','best params:', params_best,'\n')
    return params_best, trials

2 precdicting tasks

2.1 separating GBMs

In [4]:
x_1 = deepcopy(data_1)
y_1 = x_1['GRADE']
x_1 = x_1.drop(['P_IDH','P_TERT','P_CD','P_MGMT','GRADE',"ID"], axis = 1)
y_1 = np.squeeze(np.array(y_1))
y_1[y_1 < 4] = 0
y_1[y_1 == 4] = 1

In [5]:
train_data = x_1
train_label = y_1

2.1.1 hyper-parameters tuning #1

In [6]:
space4xgc = {
    'learning_rate': hp.uniform('learning_rate', 0.01,0.3),   
    'n_estimators': hp.quniform('n_estimators',50,500,25),  
    'max_depth': hp.quniform('max_depth',1,10,1),
    'min_child_weight':hp.quniform('min_child_weight',1,6,1),
    'gamma': hp.uniform('gamma',0,0.6),
    'subsample': hp.uniform('subsample',0.5,1)   
}  

In [7]:
params_best, trials = param_hyperopt(10)

100%|████████| 10/10 [09:10<00:00, 55.04s/trial, best loss: -0.8890197730592067]

 
 best params: {'gamma': 0.1661546605508867, 'learning_rate': 0.17495447949997875, 'max_depth': 3.0, 'min_child_weight': 3.0, 'n_estimators': 150.0, 'subsample': 0.8952891866184292} 



In [8]:
params_best, trials = param_hyperopt(10)

100%|████████| 10/10 [08:23<00:00, 50.39s/trial, best loss: -0.9005089315807213]

 
 best params: {'gamma': 0.3769476768334515, 'learning_rate': 0.045097589289080545, 'max_depth': 3.0, 'min_child_weight': 4.0, 'n_estimators': 150.0, 'subsample': 0.8086473131183209} 



In [9]:
params_best, trials = param_hyperopt(10)

100%|████████| 10/10 [07:35<00:00, 45.57s/trial, best loss: -0.9019093360296595]

 
 best params: {'gamma': 0.5346742325587613, 'learning_rate': 0.03346495465894004, 'max_depth': 1.0, 'min_child_weight': 5.0, 'n_estimators': 425.0, 'subsample': 0.6921673929430531} 



In [10]:
space4xgc = {
    'learning_rate': hp.uniform('learning_rate', 0.01,0.1),   
    'n_estimators': hp.quniform('n_estimators',100,500,25),  
    'max_depth': hp.quniform('max_depth',1,4,1),
    'min_child_weight':hp.quniform('min_child_weight',1,6,1),
    'gamma': hp.uniform('gamma',0,0.6),
    'subsample': hp.uniform('subsample',0.6,1)   
}  

In [11]:
params_best, trials = param_hyperopt(100)

100%|████| 100/100 [1:21:38<00:00, 48.98s/trial, best loss: -0.9029833726547579]

 
 best params: {'gamma': 0.496047986952856, 'learning_rate': 0.037795055043222656, 'max_depth': 1.0, 'min_child_weight': 5.0, 'n_estimators': 500.0, 'subsample': 0.7460511172434953} 



In [12]:
params_best, trials = param_hyperopt(100)

100%|████| 100/100 [1:16:46<00:00, 46.06s/trial, best loss: -0.902222783956859]

 
 best params: {'gamma': 0.4686447266929913, 'learning_rate': 0.04613166670340646, 'max_depth': 2.0, 'min_child_weight': 4.0, 'n_estimators': 200.0, 'subsample': 0.6440900269661416} 



In [13]:
params_best, trials = param_hyperopt(100)

100%|████| 100/100 [2:18:49<00:00, 83.30s/trial, best loss: -0.9026423997303674]

 
 best params: {'gamma': 0.3255986198986504, 'learning_rate': 0.07648788984235502, 'max_depth': 1.0, 'min_child_weight': 5.0, 'n_estimators': 275.0, 'subsample': 0.761400895661249} 



In [14]:
space4xgc = {
    'learning_rate': hp.uniform('learning_rate', 0.01,0.5),   
    'n_estimators': hp.quniform('n_estimators',250,600,25),  
    'max_depth': 1, #hp.quniform('max_depth',1,10,1),
    'min_child_weight': hp.quniform('min_child_weight',4,7,1),
    'gamma': hp.uniform('gamma',0.4,0.7),
    'subsample': hp.uniform('subsample',0.7,1)   
}  

In [15]:
params_best, trials = param_hyperopt(100)

100%|████| 100/100 [1:09:56<00:00, 41.97s/trial, best loss: -0.9029833726547579]

 
 best params: {'gamma': 0.496047986952856, 'learning_rate': 0.037795055043222656, 'min_child_weight': 5.0, 'n_estimators': 500.0, 'subsample': 0.7460511172434953} 



In [16]:
params_best, trials = param_hyperopt(100)

100%|████| 100/100 [1:12:52<00:00, 43.73s/trial, best loss: -0.9028367599146163]

 
 best params: {'gamma': 0.6527190548035862, 'learning_rate': 0.07856894579854146, 'min_child_weight': 6.0, 'n_estimators': 250.0, 'subsample': 0.8248093602169758} 



In [17]:
params_best, trials = param_hyperopt(100)

100%|████| 100/100 [1:25:06<00:00, 51.07s/trial, best loss: -0.9028648466464442]

 
 best params: {'gamma': 0.4266384897455767, 'learning_rate': 0.03835493626905805, 'min_child_weight': 7.0, 'n_estimators': 500.0, 'subsample': 0.9400109522928501} 



In [18]:
space4xgc = {
    'learning_rate': hp.uniform('learning_rate', 0.03,0.07),   
    'n_estimators': hp.quniform('n_estimators',475,525,5),  
    'max_depth': 1, #hp.quniform('max_depth',1,10,1),
    'min_child_weight': 6, #hp.quniform('min_child_weight',1,6,1),
    'gamma': hp.uniform('gamma',0.4,0.7),
    'subsample': hp.uniform('subsample',0.8,1)   
}  

In [19]:
params_best, trials = param_hyperopt(100)

100%|████| 100/100 [1:51:39<00:00, 66.99s/trial, best loss: -0.9031238063138973]

 
 best params: {'gamma': 0.6422643498766275, 'learning_rate': 0.046905106038914096, 'n_estimators': 500.0, 'subsample': 0.8895201827332386} 



In [20]:
space4xgc = {
    'learning_rate': hp.uniform('learning_rate', 0.03,0.07),   
    'n_estimators': hp.quniform('n_estimators',475,525,5),  
    'max_depth': 1, #hp.quniform('max_depth',1,10,1),
    'min_child_weight': 6, #hp.quniform('min_child_weight',1,6,1),
    'gamma': hp.uniform('gamma',0.4,0.7),
    'subsample': 1#hp.uniform('subsample',0.8,1)   
}  

In [21]:
params_best, trials = param_hyperopt(100)

100%|█████| 100/100 [1:40:17<00:00, 60.18s/trial, best loss: -0.9031035838669814]

 
 best params: {'gamma': 0.5587165700263775, 'learning_rate': 0.05347630328039607, 'n_estimators': 500.0} 



In [22]:
feature_set_0 = train_data.columns
clf_0 = XGBClassifier(n_estimators = 500, max_depth = 1, learning_rate = 0.046905106038914096, min_child_weight = 6,
                            gamma = 0.6422643498766275, subsample = 0.8895201827332386, random_state = 123)

2.1.2 feature selection

In [23]:
''' 7000HOCV_0: feature_importance rank '''
auc_0 = [] 
fea_imp_0 = []
for r in [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]:
    for n in range(1000):
        x_train, x_val, y_train, y_val = train_test_split(train_data, train_label, train_size = r, random_state= n, stratify = train_label)
        clf_0.fit(x_train, y_train)
        # print(len(x_train))
        fea_imp_0.append(list(clf_0.feature_importances_))
        probs = clf_0.predict_proba(x_val)
        auc_n = metrics.roc_auc_score(y_val, probs[:, 1])
        auc_0.append(auc_n)
    print(r)
print('mean auc in 7000HOCV_0: ', np.mean(auc_0))

0.8
0.7
0.6
0.5
0.4
0.3
0.2
mean auc in 7000HOCV_0:  0.890203425492963


In [24]:
fea_imp_0 = pd.DataFrame(fea_imp_0)
fea_imp_0.columns = feature_set_0

d_fi_0 = fea_imp_0.describe()
d_fi_0 = d_fi_0.sort_values(by = 'mean', axis = 1, ascending = False)
fea_set_0_1 = d_fi_0.columns # features ranked in descending order by the mean value of feature importances in the 7000 procedures.

fea_imp_1 = fea_imp_0[fea_set_0_1]
fea_imp_1 = [fea_imp_1 > 0]
fea_imp_1 = fea_imp_1[0]
fea_imp_1 += 0
# fea_imp_1 : procedures in which features gained importance

d_fea_imp_1 = fea_imp_1.describe()
feature_set_0_1 = fea_set_0_1[d_fea_imp_1.loc['mean', :] > 0.85] # features gained importance in more than 85% of the 7000 procedures

d_fi_0[feature_set_0_1] 

,S_NUM_S,S_BTH_T_RA,AGE,S_E_T,S_BTH_T,S_T,F,I,S_ET_RA,S_DT,SEX,S_ET,S_ABD_T,S_DT_RA,S_ST
count,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000
mean,0.149396,0.148437,0.059370,0.050503,0.040706,0.035231,0.027333,0.026048,0.024756,0.018503,0.017284,0.013315,0.013009,0.011839,0.011828
std,0.046113,0.058571,0.014645,0.028845,0.039283,0.010500,0.010073,0.012250,0.011755,0.009603,0.005745,0.004146,0.011554,0.004829,0.005117
min,0.000000,0.000000,0.022179,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.119295,0.111531,0.050484,0.029719,0.013240,0.028500,0.021370,0.020057,0.017057,0.012618,0.013812,0.010754,0.008834,0.009152,0.008763
50%,0.146773,0.153922,0.056867,0.047900,0.024707,0.034490,0.025883,0.023801,0.022242,0.015925,0.016195,0.012643,0.011238,0.010856,0.010567
75%,0.178626,0.191496,0.064696,0.066821,0.055426,0.040519,0.031370,0.028177,0.028958,0.021441,0.019467,0.015044,0.014569,0.013374,0.013691
max,0.338440,0.368119,0.168841,0.237773,0.335792,0.115012,0.100463,0.156574,0.140852,0.117455,0.057375,0.114479,0.247700,0.066003,0.088448


In [25]:
''' 7000HOCV_1*10: feature_importance rank '''
fea_set_0_2 = deepcopy(feature_set_0_1)
auc_mean_1_1 = [] 
for i in range(13):
    i += 2
    test_feas = fea_set_0_2[:i]
    auc_i = []
    for r in [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]:
        for n in range(1000):
            x_train, x_val, y_train, y_val = train_test_split(train_data[test_feas], train_label, train_size = r, random_state= n, stratify = train_label)
            clf_0.fit(x_train, y_train)
            probs = clf_0.predict_proba(x_val)
            auc_n = metrics.roc_auc_score(y_val, probs[:, 1])
            auc_i.append(auc_n)
        print(r)
    auc_mean_1_1.append(np.mean(auc_i)) 
    print('i = ', i, '\ntest_feas:', test_feas, '\nauc_m = ', np.mean(auc_i))


0.8
0.7
0.6
0.5
0.4
0.3
0.2
i =  2 
test_feas: Index(['S_NUM_S', 'S_BTH_T_RA'], dtype='object') 
auc_m =  0.7863852340817477
0.8
0.7
0.6
0.5
0.4
0.3
0.2
i =  3 
test_feas: Index(['S_NUM_S', 'S_BTH_T_RA', 'AGE'], dtype='object') 
auc_m =  0.8579622910251392
0.8
0.7
0.6
0.5
0.4
0.3
0.2
i =  4 
test_feas: Index(['S_NUM_S', 'S_BTH_T_RA', 'AGE', 'S_E_T'], dtype='object') 
auc_m =  0.8610177413142968
0.8
0.7
0.6
0.5
0.4
0.3
0.2
i =  5 
test_feas: Index(['S_NUM_S', 'S_BTH_T_RA', 'AGE', 'S_E_T', 'S_BTH_T'], dtype='object') 
auc_m =  0.864933654674168
0.8
0.7
0.6
0.5
0.4
0.3
0.2
i =  6 
test_feas: Index(['S_NUM_S', 'S_BTH_T_RA', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T'], dtype='object') 
auc_m =  0.8693048192947748
0.8
0.7
0.6
0.5
0.4
0.3
0.2
i =  7 
test_feas: Index(['S_NUM_S', 'S_BTH_T_RA', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F'], dtype='object') 
auc_m =  0.8745669492880458
0.8
0.7
0.6
0.5
0.4
0.3
0.2
i =  8 
test_feas: Index(['S_NUM_S', 'S_BTH_T_RA', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I'], dtyp

In [26]:
test_feas = fea_set_0_2
auc_i = []
for r in [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]:
    for n in range(1000):
        x_train, x_val, y_train, y_val = train_test_split(train_data[test_feas], train_label, train_size = r, random_state= n, stratify = train_label)
        clf_0.fit(x_train, y_train)
        probs = clf_0.predict_proba(x_val)
        auc_n = metrics.roc_auc_score(y_val, probs[:, 1])
        auc_i.append(auc_n)
    print(r)
auc_mean_1_1.append(np.mean(auc_i)) 
print('ntest_feas:', test_feas, '\nauc_m = ', np.mean(auc_i))

for i in range(2):
    test_feas = fea_set_0_2[i:]
    auc_i = []
    for r in [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]:
        for n in range(1000):
            x_train, x_val, y_train, y_val = train_test_split(train_data[test_feas], train_label, train_size = r, random_state= n, stratify = train_label)
            clf_0.fit(x_train, y_train)
            probs = clf_0.predict_proba(x_val)
            auc_n = metrics.roc_auc_score(y_val, probs[:, 1])
            auc_i.append(auc_n)
        print(r)
    auc_mean_1_1.append(np.mean(auc_i)) 
    print('i = ', i, '\ntest_feas:', test_feas, '\nauc_m = ', np.mean(auc_i))

idx_1 = list(range(15))
idx_1.remove[1]
test_feas = fea_set_0_2[idx_1]
auc_i = []
for r in [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]:
    for n in range(1000):
        x_train, x_val, y_train, y_val = train_test_split(train_data[test_feas], train_label, train_size = r, random_state= n, stratify = train_label)
        clf_0.fit(x_train, y_train)
        probs = clf_0.predict_proba(x_val)
        auc_n = metrics.roc_auc_score(y_val, probs[:, 1])
        auc_i.append(auc_n)
    print(r)
auc_mean_1_1.append(np.mean(auc_i)) 
print('test_feas:', test_feas, '\nauc_m = ', np.mean(auc_i))

0.8
0.7
0.6
0.5
0.4
0.3
0.2
ntest_feas: Index(['S_NUM_S', 'S_BTH_T_RA', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I',
       'S_ET_RA', 'S_DT', 'SEX', 'S_ET', 'S_ABD_T', 'S_DT_RA', 'S_ST'],
      dtype='object') 
auc_m =  0.8891414129155955
0.8
0.7
0.6
0.5
0.4
0.3
0.2
i =  0 
test_feas: Index(['S_NUM_S', 'S_BTH_T_RA', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I',
       'S_ET_RA', 'S_DT', 'SEX', 'S_ET', 'S_ABD_T', 'S_DT_RA', 'S_ST'],
      dtype='object') 
auc_m =  0.8891414129155955
0.8
0.7
0.6
0.5
0.4
0.3
0.2
i =  1 
test_feas: Index(['S_BTH_T_RA', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I', 'S_ET_RA',
       'S_DT', 'SEX', 'S_ET', 'S_ABD_T', 'S_DT_RA', 'S_ST'],
      dtype='object') 
auc_m =  0.8835183593244517


TypeError: 'builtin_function_or_method' object is not subscriptable

In [27]:
auc_mean_1_1.remove(auc_mean_1_1[-2]) #duplicated result of  'S_BTH_T_RA', 'S_NUM_S', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I','S_ET_RA', 'S_DT', 'SEX', 'S_ET', 'S_ABD_T', 'S_DT_RA', 'S_ST'
test_feas = fea_set_0_2[2:]
auc_i = []
for r in [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]:
    for n in range(1000):
        x_train, x_val, y_train, y_val = train_test_split(train_data[test_feas], train_label, train_size = r, random_state= n, stratify = train_label)
        clf_0.fit(x_train, y_train)
        probs = clf_0.predict_proba(x_val)
        auc_n = metrics.roc_auc_score(y_val, probs[:, 1])
        auc_i.append(auc_n)
    print(r)
auc_mean_1_1.append(np.mean(auc_i)) 
print('test_feas:', test_feas, '\nauc_m = ', np.mean(auc_i))

idx_1 = list(range(15))
idx_1.remove(1)
test_feas = fea_set_0_2[idx_1]
auc_i = []
for r in [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]:
    for n in range(1000):
        x_train, x_val, y_train, y_val = train_test_split(train_data[test_feas], train_label, train_size = r, random_state= n, stratify = train_label)
        clf_0.fit(x_train, y_train)
        probs = clf_0.predict_proba(x_val)
        auc_n = metrics.roc_auc_score(y_val, probs[:, 1])
        auc_i.append(auc_n)
    print(r)
auc_mean_1_1.append(np.mean(auc_i)) 
print('test_feas:', test_feas, '\nauc_m = ', np.mean(auc_i))
print('auc_mean_list:', auc_mean_1_1)

0.8
0.7
0.6
0.5
0.4
0.3
0.2
test_feas: Index(['AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I', 'S_ET_RA', 'S_DT', 'SEX',
       'S_ET', 'S_ABD_T', 'S_DT_RA', 'S_ST'],
      dtype='object') 
auc_m =  0.8845504360121276
0.8
0.7
0.6
0.5
0.4
0.3
0.2
test_feas: Index(['S_NUM_S', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I', 'S_ET_RA',
       'S_DT', 'SEX', 'S_ET', 'S_ABD_T', 'S_DT_RA', 'S_ST'],
      dtype='object') 
auc_m =  0.8896686747781974
auc_mean_list: [0.7863852340817477, 0.8579622910251392, 0.8610177413142968, 0.864933654674168, 0.8693048192947748, 0.8745669492880458, 0.8813603918683424, 0.8823674521594361, 0.8814518779711444, 0.8856999505047463, 0.886651464878821, 0.8863686097599678, 0.8859584258924229, 0.8891414129155955, 0.8835183593244517, 0.8845504360121276, 0.8896686747781974]


In [28]:
test_feas = ['S_NUM_S', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I', 'S_ET_RA', 'SEX', 'S_ET', 'S_ST']
auc_i = []
for r in [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]:
    for n in range(1000):
        x_train, x_val, y_train, y_val = train_test_split(train_data[test_feas], train_label, train_size = r, random_state= n, stratify = train_label)
        clf_0.fit(x_train, y_train)
        probs = clf_0.predict_proba(x_val)
        auc_n = metrics.roc_auc_score(y_val, probs[:, 1])
        auc_i.append(auc_n)
    print(r)
auc_mean_1_1.append(np.mean(auc_i)) 
print('test_feas:', test_feas, '\nauc_m = ', np.mean(auc_i))


test_feas = ['S_NUM_S', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I', 'S_ET_RA', 'SEX', 'S_ET', 'S_ST',
            'S_BTH_T_RA']
auc_i = []
for r in [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]:
    for n in range(1000):
        x_train, x_val, y_train, y_val = train_test_split(train_data[test_feas], train_label, train_size = r, random_state= n, stratify = train_label)
        clf_0.fit(x_train, y_train)
        probs = clf_0.predict_proba(x_val)
        auc_n = metrics.roc_auc_score(y_val, probs[:, 1])
        auc_i.append(auc_n)
    print(r)
auc_mean_1_1.append(np.mean(auc_i)) 
print('test_feas:', test_feas, '\nauc_m = ', np.mean(auc_i))

test_feas = ['S_NUM_S', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I', 'S_ET_RA', 'SEX', 'S_ET', 'S_ST'
            'S_DT']

auc_i = []
for r in [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]:
    for n in range(1000):
        x_train, x_val, y_train, y_val = train_test_split(train_data[test_feas], train_label, train_size = r, random_state= n, stratify = train_label)
        clf_0.fit(x_train, y_train)
        probs = clf_0.predict_proba(x_val)
        auc_n = metrics.roc_auc_score(y_val, probs[:, 1])
        auc_i.append(auc_n)
    print(r)
auc_mean_1_1.append(np.mean(auc_i)) 
print('test_feas:', test_feas, '\nauc_m = ', np.mean(auc_i))


test_feas = ['S_NUM_S', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I', 'S_ET_RA', 'SEX', 'S_ET', 'S_ST'
            'S_ABD_T']

auc_i = []
for r in [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]:
    for n in range(1000):
        x_train, x_val, y_train, y_val = train_test_split(train_data[test_feas], train_label, train_size = r, random_state= n, stratify = train_label)
        clf_0.fit(x_train, y_train)
        probs = clf_0.predict_proba(x_val)
        auc_n = metrics.roc_auc_score(y_val, probs[:, 1])
        auc_i.append(auc_n)
    print(r)
auc_mean_1_1.append(np.mean(auc_i)) 
print('test_feas:', test_feas, '\nauc_m = ', np.mean(auc_i))


test_feas = ['S_NUM_S', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I', 'S_ET_RA', 'SEX', 'S_ET', 'S_ST'
            'S_DT_RA']

auc_i = []
for r in [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]:
    for n in range(1000):
        x_train, x_val, y_train, y_val = train_test_split(train_data[test_feas], train_label, train_size = r, random_state= n, stratify = train_label)
        clf_0.fit(x_train, y_train)
        probs = clf_0.predict_proba(x_val)
        auc_n = metrics.roc_auc_score(y_val, probs[:, 1])
        auc_i.append(auc_n)
    print(r)
auc_mean_1_1.append(np.mean(auc_i)) 
print('test_feas:', test_feas, '\nauc_m = ', np.mean(auc_i))




0.8
0.7
0.6
0.5
0.4
0.3
0.2
test_feas: ['S_NUM_S', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I', 'S_ET_RA', 'SEX', 'S_ET', 'S_ST'] 
auc_m =  0.89120162075546
0.8
0.7
0.6
0.5
0.4
0.3
0.2
test_feas: ['S_NUM_S', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I', 'S_ET_RA', 'SEX', 'S_ET', 'S_ST', 'S_BTH_T_RA'] 
auc_m =  0.8905891929692805


KeyError: "['S_STS_DT'] not in index"

In [29]:
test_feas = ['S_NUM_S', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I', 'S_ET_RA', 'SEX', 'S_ET', 'S_ST',
            'S_DT']

auc_i = []
for r in [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]:
    for n in range(1000):
        x_train, x_val, y_train, y_val = train_test_split(train_data[test_feas], train_label, train_size = r, random_state= n, stratify = train_label)
        clf_0.fit(x_train, y_train)
        probs = clf_0.predict_proba(x_val)
        auc_n = metrics.roc_auc_score(y_val, probs[:, 1])
        auc_i.append(auc_n)
    print(r)
auc_mean_1_1.append(np.mean(auc_i)) 
print('test_feas:', test_feas, '\nauc_m = ', np.mean(auc_i))


test_feas = ['S_NUM_S', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I', 'S_ET_RA', 'SEX', 'S_ET', 'S_ST',
            'S_ABD_T']

auc_i = []
for r in [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]:
    for n in range(1000):
        x_train, x_val, y_train, y_val = train_test_split(train_data[test_feas], train_label, train_size = r, random_state= n, stratify = train_label)
        clf_0.fit(x_train, y_train)
        probs = clf_0.predict_proba(x_val)
        auc_n = metrics.roc_auc_score(y_val, probs[:, 1])
        auc_i.append(auc_n)
    print(r)
auc_mean_1_1.append(np.mean(auc_i)) 
print('test_feas:', test_feas, '\nauc_m = ', np.mean(auc_i))


test_feas = ['S_NUM_S', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I', 'S_ET_RA', 'SEX', 'S_ET', 'S_ST',
            'S_DT_RA']

auc_i = []
for r in [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]:
    for n in range(1000):
        x_train, x_val, y_train, y_val = train_test_split(train_data[test_feas], train_label, train_size = r, random_state= n, stratify = train_label)
        clf_0.fit(x_train, y_train)
        probs = clf_0.predict_proba(x_val)
        auc_n = metrics.roc_auc_score(y_val, probs[:, 1])
        auc_i.append(auc_n)
    print(r)
auc_mean_1_1.append(np.mean(auc_i)) 
print('test_feas:', test_feas, '\nauc_m = ', np.mean(auc_i))


0.8
0.7
0.6
0.5
0.4
0.3
0.2
test_feas: ['S_NUM_S', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I', 'S_ET_RA', 'SEX', 'S_ET', 'S_ST', 'S_DT'] 
auc_m =  0.8904548879331592
0.8
0.7
0.6
0.5
0.4
0.3
0.2
test_feas: ['S_NUM_S', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I', 'S_ET_RA', 'SEX', 'S_ET', 'S_ST', 'S_ABD_T'] 
auc_m =  0.8907341756315349
0.8
0.7
0.6
0.5
0.4
0.3
0.2
test_feas: ['S_NUM_S', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I', 'S_ET_RA', 'SEX', 'S_ET', 'S_ST', 'S_DT_RA'] 
auc_m =  0.8902549717690682


2.1.3 hyper-parameters tuning #2

In [30]:
'''
clf_0 = XGBClassifier(n_estimators = 500, max_depth = 1, learning_rate = 0.046905106038914096, min_child_weight = 6,
                            gamma = 0.6422643498766275, subsample = 0.8895201827332386, random_state = 123)
'''
significant_features = ['S_NUM_S', 'AGE', 'S_E_T', 'S_BTH_T', 'S_T', 'F', 'I', 'S_ET_RA', 'SEX', 'S_ET', 'S_ST']
train_data = train_data[significant_features]
auc_i = []
for n in range(100):
    x_train, x_val, y_train, y_val=train_test_split(train_data, train_label, train_size = 0.8, random_state= n, stratify= train_label)
    clf_0.fit(x_train, y_train)
    probs = clf_0.predict_proba(x_val)
    auc_n = metrics.roc_auc_score(y_val, probs[:, 1])
    auc_i.append(auc_n)
auc_m = np.mean(auc_i)
print('mean auc of clf_0 in 100HOCV is : ', auc_m)

mean auc of clf_0 in 100HOCV is :  0.8995331985170203


In [31]:
space4xgc = {
    'learning_rate': hp.uniform('learning_rate', 0.01,0.3),   
    'n_estimators': hp.quniform('n_estimators',50,600,25),  
    'max_depth': hp.quniform('max_depth',1,10,1),
    'min_child_weight':hp.quniform('min_child_weight',1,6,1),
    'gamma': hp.uniform('gamma',0,0.6),
    'subsample': hp.uniform('subsample',0.5,1)   
}  

In [32]:
params_best, trials = param_hyperopt(100)

100%|████| 100/100 [1:36:39<00:00, 58.00s/trial, best loss: -0.9006459948320413]

 
 best params: {'gamma': 0.40945639709064485, 'learning_rate': 0.048926565156366265, 'max_depth': 1.0, 'min_child_weight': 4.0, 'n_estimators': 525.0, 'subsample': 0.6068095625863299} 



In [33]:
params_best, trials = param_hyperopt(100)

100%|██████| 100/100 [55:42<00:00, 33.43s/trial, best loss: -0.9000938096843052]

 
 best params: {'gamma': 0.04465803117963073, 'learning_rate': 0.13910537623047972, 'max_depth': 1.0, 'min_child_weight': 3.0, 'n_estimators': 150.0, 'subsample': 0.5832982508843672} 



In [34]:
params_best, trials = param_hyperopt(100)

100%|████| 100/100 [1:28:05<00:00, 52.85s/trial, best loss: -0.9003595101673969]

 
 best params: {'gamma': 0.3015531478209838, 'learning_rate': 0.06102109916148678, 'max_depth': 1.0, 'min_child_weight': 4.0, 'n_estimators': 450.0, 'subsample': 0.6956124975796463} 



In [35]:
space4xgc = {
    'learning_rate': hp.uniform('learning_rate', 0.01,0.1),   
    'n_estimators': hp.quniform('n_estimators',425,550,25),  
    'max_depth': 1,#hp.quniform('max_depth',1,10,1),
    'min_child_weight':4,#hp.quniform('min_child_weight',3,4,1),
    'gamma': hp.uniform('gamma',0.2,0.5),
    'subsample': hp.uniform('subsample',0.6,0.7)   
}  

In [36]:
params_best, trials = param_hyperopt(100)

100%|████| 100/100 [1:36:51<00:00, 58.12s/trial, best loss: -0.9008336142006516]

 
 best params: {'gamma': 0.34372492822303385, 'learning_rate': 0.046262676681857046, 'n_estimators': 500.0, 'subsample': 0.6000492296088057} 



In [37]:
params_best, trials = param_hyperopt(100)

100%|████| 100/100 [1:25:08<00:00, 51.08s/trial, best loss: -0.9008324907313786]

 
 best params: {'gamma': 0.27854602115869626, 'learning_rate': 0.04078111435633796, 'n_estimators': 550.0, 'subsample': 0.6102454595456299} 



In [38]:
params_best, trials = param_hyperopt(100)

100%|████| 100/100 [1:21:42<00:00, 49.02s/trial, best loss: -0.9008072126727333]

 
 best params: {'gamma': 0.3078956240841772, 'learning_rate': 0.043135595181805354, 'n_estimators': 525.0, 'subsample': 0.6054642007794796} 



In [39]:
space4xgc = {
    'learning_rate': hp.uniform('learning_rate', 0.035,0.05),   
    'n_estimators': hp.quniform('n_estimators',475,575,5),  
    'max_depth': 1,#hp.quniform('max_depth',1,10,1),
    'min_child_weight':4,#hp.quniform('min_child_weight',3,4,1),
    'gamma': hp.uniform('gamma',0.2,0.4),
    'subsample': hp.uniform('subsample',0.55,0.65)   
}  

In [40]:
params_best, trials = param_hyperopt(100)

100%|████| 100/100 [1:35:49<00:00, 57.50s/trial, best loss: -0.9009802269407934]

 
 best params: {'gamma': 0.2508120619766673, 'learning_rate': 0.039015041456294926, 'n_estimators': 540.0, 'subsample': 0.5675994149060832} 



In [41]:
params_best, trials = param_hyperopt(100)

100%|█████| 100/100 [1:37:55<00:00, 58.76s/trial, best loss: -0.900993146837434]

 
 best params: {'gamma': 0.26411450690027444, 'learning_rate': 0.044912211652964766, 'n_estimators': 540.0, 'subsample': 0.5565227602988679} 



In [42]:
space4xgc = {
    'learning_rate': hp.uniform('learning_rate', 0.035,0.05),   
    'n_estimators': hp.quniform('n_estimators',500,550,5),  
    'max_depth': 1,#hp.quniform('max_depth',1,10,1),
    'min_child_weight':4,#hp.quniform('min_child_weight',3,4,1),
    'gamma': hp.uniform('gamma',0.2,0.3),
    'subsample': hp.uniform('subsample',0.55,0.6)   
}  

In [43]:
params_best, trials = param_hyperopt(100)

100%|████| 100/100 [1:57:57<00:00, 70.77s/trial, best loss: -0.9009976407145266]

 
 best params: {'gamma': 0.26297758009521893, 'learning_rate': 0.04000290859957704, 'n_estimators': 520.0, 'subsample': 0.5662940473330792} 



In [44]:
params_best, trials = param_hyperopt(100)

100%|████| 100/100 [1:56:13<00:00, 69.73s/trial, best loss: -0.9010088754072576]

 
 best params: {'gamma': 0.20811750149748293, 'learning_rate': 0.0403205032029171, 'n_estimators': 535.0, 'subsample': 0.5649957071497334} 



2.2 prediction for IDH status (failed)

In [45]:
x_2 = deepcopy(data_1)
x_2 = x_2[x_2['P_IDH'] > 0]
x_2.index = range(len(x_2))
y_2 = x_2['P_IDH']
x_2 = x_2.drop(['P_IDH','P_TERT','P_CD','P_MGMT','GRADE',"ID"], axis = 1)
y_2 = np.squeeze(np.array(y_2))
y_2[y_2 == 1] = 0
y_2[y_2 == 2] = 1

In [46]:
train_data = x_2
train_label = y_2

In [47]:
space4xgc = {
    'learning_rate': hp.uniform('learning_rate', 0.01,0.3),   
    'n_estimators': hp.quniform('n_estimators',50,500,25),  
    'max_depth': hp.quniform('max_depth',1,10,1),
    'min_child_weight':hp.quniform('min_child_weight',1,6,1),
    'gamma': hp.uniform('gamma',0,0.6),
    'subsample': hp.uniform('subsample',0.5,1)   
}  

In [48]:
params_best, trials = param_hyperopt(10)

100%|████████| 10/10 [07:25<00:00, 44.59s/trial, best loss: -0.8313802622498275]

 
 best params: {'gamma': 0.12253060002096547, 'learning_rate': 0.03809913109189958, 'max_depth': 8.0, 'min_child_weight': 2.0, 'n_estimators': 75.0, 'subsample': 0.538379117989098} 



In [49]:
params_best, trials = param_hyperopt(10)

100%|████████| 10/10 [07:35<00:00, 45.57s/trial, best loss: -0.8297722567287784]

 
 best params: {'gamma': 0.41344046447972377, 'learning_rate': 0.02566389009868622, 'max_depth': 2.0, 'min_child_weight': 5.0, 'n_estimators': 75.0, 'subsample': 0.6369062878868266} 



In [50]:
params_best, trials = param_hyperopt(10)

100%|████████| 10/10 [08:03<00:00, 48.36s/trial, best loss: -0.8175914423740511]

 
 best params: {'gamma': 0.2935409265994895, 'learning_rate': 0.06841295278193911, 'max_depth': 2.0, 'min_child_weight': 6.0, 'n_estimators': 275.0, 'subsample': 0.6500372165227859} 



In [51]:
params_best, trials = param_hyperopt(100)

100%|████| 100/100 [1:00:47<00:00, 36.47s/trial, best loss: -0.8381815044858522]

 
 best params: {'gamma': 0.4936180366704832, 'learning_rate': 0.05390306073762062, 'max_depth': 1.0, 'min_child_weight': 4.0, 'n_estimators': 175.0, 'subsample': 0.7932926409179585} 



2.3 prediction for 1p/19q status (failed)

In [52]:
x_3 = deepcopy(data_1)
x_3 = x_3[x_3['P_IDH'] == 1][x_3['P_CD'] > 0]
x_3.index = range(len(x_3))
y_3 = x_3['P_CD']
x_3 = x_3.drop(['P_IDH','P_TERT','P_CD','P_MGMT','GRADE',"ID"], axis = 1)
y_3 = np.squeeze(np.array(y_3))
y_3[y_3 == 1] = 0
y_3[y_3 == 2] = 1

In [53]:
params_best, trials = param_hyperopt(10)

 20%|█▊       | 2/10 [02:41<10:46, 80.86s/trial, best loss: -0.8082677708764663]


KeyboardInterrupt: 

In [54]:
train_data = x_3
train_label = y_3

In [55]:
params_best, trials = param_hyperopt(10)

100%|███████| 10/10 [08:23<00:00, 50.38s/trial, best loss: -0.47173387096774194]

 
 best params: {'gamma': 0.4332366324153875, 'learning_rate': 0.10794813777212135, 'max_depth': 7.0, 'min_child_weight': 1.0, 'n_estimators': 500.0, 'subsample': 0.5859849110807285} 



In [56]:
params_best, trials = param_hyperopt(10)

100%|█████████| 10/10 [05:17<00:00, 31.73s/trial, best loss: -0.494502688172043]

 
 best params: {'gamma': 0.18479344789450414, 'learning_rate': 0.017814347320118266, 'max_depth': 1.0, 'min_child_weight': 4.0, 'n_estimators': 150.0, 'subsample': 0.5156562706511524} 



In [57]:
params_best, trials = param_hyperopt(10)

100%|████████| 10/10 [05:39<00:00, 33.99s/trial, best loss: -0.4716263440860216]

 
 best params: {'gamma': 0.47624865538492667, 'learning_rate': 0.278750253822811, 'max_depth': 5.0, 'min_child_weight': 1.0, 'n_estimators': 250.0, 'subsample': 0.6981003931199743} 



2.4 prediction for TERT status (failed)

In [58]:
x_4 = deepcopy(data_1)
x_4 = x_4[x_4['P_TERT'] > 0]
x_4.index = range(len(x_4))
y_4 = x_4['P_TERT']
x_4 = x_4.drop(['P_IDH','P_TERT','P_CD','P_MGMT','GRADE',"ID"], axis = 1)
y_4 = np.squeeze(np.array(y_4))
y_4[y_4 == 1] = 0
y_4[y_4 == 2] = 1

In [59]:
train_data = x_4
train_label = y_4

In [60]:
params_best, trials = param_hyperopt(10)

100%|████████| 10/10 [06:38<00:00, 39.88s/trial, best loss: -0.6106820276497696]

 
 best params: {'gamma': 0.599357404691213, 'learning_rate': 0.014413601119044581, 'max_depth': 6.0, 'min_child_weight': 1.0, 'n_estimators': 75.0, 'subsample': 0.6106826819891433} 



In [61]:
params_best, trials = param_hyperopt(10)

100%|████████| 10/10 [06:48<00:00, 40.85s/trial, best loss: -0.5896036866359445]

 
 best params: {'gamma': 0.4393447762800538, 'learning_rate': 0.21035107231581682, 'max_depth': 5.0, 'min_child_weight': 1.0, 'n_estimators': 475.0, 'subsample': 0.7215685413313747} 



In [62]:
params_best, trials = param_hyperopt(10)

100%|████████| 10/10 [06:37<00:00, 39.70s/trial, best loss: -0.5987142857142858]

 
 best params: {'gamma': 0.11767204659033656, 'learning_rate': 0.019490298664433442, 'max_depth': 2.0, 'min_child_weight': 2.0, 'n_estimators': 325.0, 'subsample': 0.8538182550556355} 



2.5 prediction for MGMT status (failed)

In [63]:
x_5 = deepcopy(data_1)
x_5 = x_5[x_5['P_MGMT'] > 0]
x_5.index = range(len(x_5))
y_5 = x_5['P_MGMT']
x_5 = x_5.drop(['P_IDH','P_TERT','P_CD','P_MGMT','GRADE',"ID"], axis = 1)
y_5 = np.squeeze(np.array(y_5))
y_5[y_5 == 1] = 0
y_5[y_5 == 2] = 1

In [64]:
train_data = x_5
train_label = y_5

In [65]:
params_best, trials = param_hyperopt(10)

100%|███████| 10/10 [06:47<00:00, 40.70s/trial, best loss: -0.48292778649921503]

 
 best params: {'gamma': 0.005867192433231949, 'learning_rate': 0.28741326669155925, 'max_depth': 9.0, 'min_child_weight': 1.0, 'n_estimators': 300.0, 'subsample': 0.8318538599908324} 



In [66]:
params_best, trials = param_hyperopt(10)

100%|███████| 10/10 [05:11<00:00, 31.17s/trial, best loss: -0.48737964416535845]

 
 best params: {'gamma': 0.28250579572544626, 'learning_rate': 0.20688823394675318, 'max_depth': 1.0, 'min_child_weight': 2.0, 'n_estimators': 75.0, 'subsample': 0.9164656048405685} 



In [67]:
params_best, trials = param_hyperopt(10)

100%|███████| 10/10 [08:28<00:00, 50.82s/trial, best loss: -0.48379644165358454]

 
 best params: {'gamma': 0.4744922642324027, 'learning_rate': 0.14882970833508008, 'max_depth': 7.0, 'min_child_weight': 2.0, 'n_estimators': 350.0, 'subsample': 0.9561533859802508} 

